<a href="https://colab.research.google.com/github/ydaigo/ObjectDetectionColab/blob/master/ObjectDetectionColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#セットアップ
実行に10分ほどかかります。

In [0]:
%%time
%reload_ext autoreload
%autoreload 2
%matplotlib inline
!git clone https://github.com/ydaigo/ObjectDetectionColab.git
!sh /content/ObjectDetectionColab/setup.sh
%cd /content/ObjectDetectionColab/mmdetection
import os
from mmdet.apis import init_detector, inference_detector
# Standard output of images from mmdetection is slightly rewritten for some flexibility
from utils.viz import show_result
config_path = 'configs/'
os.makedirs(os.path.dirname(config_path), exist_ok=True)
model_path = 'models/'
os.makedirs(os.path.dirname(model_path), exist_ok=True)
img_path = 'imgs/'
os.makedirs(os.path.dirname(img_path), exist_ok=True)

#物体検出

In [0]:
#テスト画像のダウンロード
!wget https://t2.ea.ltmcdn.com/en/images/8/7/3/img_5_hairless_cat_breeds_2378_600.jpg -O imgs/sphynx.png
!wget https://economia.me/i/2019/03/appliances-cabin-island-house-modern-kitchen-and-gloss-cabinets-cottage-pictures-da-color-white-rustic-small-wood-country-design-photos-beach-floors-grey-images-log-decorating.jpg -O imgs/kitchen.jpg
#重みのダウンロード
!wget https://s3.ap-northeast-2.amazonaws.com/open-mmlab/mmdetection/models/faster_rcnn_x101_64x4d_fpn_1x_20181218-c9c69c8f.pth -O models/faster_rcnn_x101_64x4d_fpn_1x.pth
!wget https://s3.ap-northeast-2.amazonaws.com/open-mmlab/mmdetection/models/mask_rcnn_x101_64x4d_fpn_1x_20181218-cb159987.pth -O models/mask_rcnn_x101_64x4d_fpn_1x.pth

In [0]:
#モデルのロード
config_file = config_path+'faster_rcnn_x101_64x4d_fpn_1x.py'
checkpoint_file = model_path+'faster_rcnn_x101_64x4d_fpn_1x.pth'
model = init_detector(config_file, checkpoint_file, device='cuda:0')

In [0]:
#推論1
img = 'imgs/sphynx.png'
result = inference_detector(model, img)
show_result(img, result, model.CLASSES, score_thr=0.7, font_scale=.6, thickness=1, show_mask=False)

In [0]:
#推論２
img = 'imgs/kitchen.jpg'
result = inference_detector(model, img)
show_result(img, result, model.CLASSES, score_thr=0.5, font_scale=.6, thickness=2, show_mask=False)

#物体領域抽出(セマンティックセグメンテーション)

In [0]:
#モデルのロード
config_file = config_path+'mask_rcnn_x101_64x4d_fpn_1x.py'
checkpoint_file = model_path+'mask_rcnn_x101_64x4d_fpn_1x.pth'
model = init_detector(config_file, checkpoint_file, device='cuda:0')

In [0]:
#推論
img = 'imgs/kitchen.jpg'
result = inference_detector(model, img)
show_result(img, result, model.CLASSES, score_thr=0.5, font_scale=.6, thickness=2, show_mask=True)

#転移学習

In [0]:
%cd /content/ObjectDetectionColab/mmdetection
from utils.make_annot import annot
label_dict = {'Panda':1}
path_to_OID_train_imgs = '/content/ObjectDetectionColab/mmdetection/OIDv4_ToolKit/OID/Dataset/train/Panda/'
path_to_OID_valid_imgs = '/content/ObjectDetectionColab/mmdetection/OIDv4_ToolKit/OID/Dataset/validation/Panda/'

#pklファイルの作成
annot(path_to_OID_train_imgs, label_dict, train_data=True)
annot(path_to_OID_valid_imgs, label_dict, train_data=False)
import pickle
with open('annotations_val.pkl', 'rb') as f:
    x = pickle.load(f)

In [0]:
#学習
#34分かかる
%%time
!chmod +x ./tools/dist_train.sh
%cd /content/ObjectDetectionColab/mmdetection
!tools/dist_train.sh \
configs/faster_rcnn_r50_fpn_1x_our_version.py 1 --validate

In [0]:
!wget https://www.aws-s.com/getImage.php?path=eve053_l1.jpg -O imgs/pandas.jpg
!wget https://github.com/ydaigo/ObjectDetectionColab/releases/download/a/panda.pth -O models/panda12.pth

In [0]:
from mmdet.apis import init_detector, inference_detector

# Standard output of images from mmdetection is slightly rewritten for some flexibility
from utils.viz import show_result

config_file = 'configs/faster_rcnn_r50_fpn_1x_our_version.py'
checkpoint_file = '/content/ObjectDetectionColab/mmdetection/models/panda12.pth'

# Loading the model from the config and saved checkpoint, inference to GPU
model = init_detector(config_file, checkpoint_file, device='cuda:0')

In [0]:
classes = ['Panda']
img = 'imgs/pandas.jpg'
result = inference_detector(model, img)
show_result(img, result, classes, score_thr=0.9, font_scale=1., thickness=2, show_mask=False)